In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from typing import Sequence

from superprimitive_fusion.scanner import (
    capture_spherical_scans,
    render_rgb_view,
    _interpolate_vertex_colors,
    triangulate_rgbd_grid,
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
    triangulate_segments,
)

In [ ]:
names = (
    ('mustard-bottle', 'mustard-bottle.obj'),
    ('table', 'table.obj'),
    # ('power-drill', 'power-drill.obj'),
    # ('bleach', 'bleach.obj'),
    # ('pitcher', 'pitcher.obj'),
    # ('mug', 'mug.obj'),
    # ('extra-large-clamp', 'extra-large-clamp-leaning.obj'),
)

meshes = dict()
for foldername,filename in names:
    print(f'Getting the {filename.split('.')[0]}')
    
    mesh = o3d.io.read_triangle_mesh(f"../data/posed-meshes/{foldername}/{filename}", enable_post_processing=True)

    bake_uv_to_vertex_colours(mesh)
    mesh.compute_vertex_normals()

    meshes[foldername] = mesh

meshlist = list(meshes.values())

Getting the mustard-bottle


In [ ]:
centres = []
for meshname, mesh in meshes.items():
    if meshname == 'table':
        continue
    centres.append(mesh.get_center())

centres = np.vstack(centres)

obj_centre = centres.mean(axis=0)

In [ ]:
mesh: o3d.geometry.TriangleMesh = meshlist
cam_centre=polar2cartesian(1.0, 30, 90)
look_dir=obj_centre
dropout_rate: float = 0.0
depth_error_std: float = 0.0
translation_error_std: float = 0.0
rotation_error_std_degs: float = 0.0
width_px: int = 360
height_px: int = 240
fov: float = 70.0
k: float = 3.0

In [ ]:
###############
# Ray casting #
###############
scene = o3d.t.geometry.RaycastingScene()
geom_ids = []
for name,mesh in meshes.items():
    id = scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))
    geom_ids.append(id)
    print(f'{name} is geom id {id}')

rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
    fov_deg=fov,
    center=list(look_dir),
    eye=list(cam_centre),
    up=[0, 0, 1],
    width_px=width_px,
    height_px=height_px,
)

ans = scene.cast_rays(rays)
t_hit = ans["t_hit"].numpy()

# Intersection metadata (gometry id + triangle id + barycentric uv + normals)
geom_ids = ans["geometry_ids"].numpy().astype(np.int16)
geom_ids[geom_ids > 1e2] = -1
prim_ids = ans["primitive_ids"].numpy().astype(np.int32)
bary_uv = ans.get("primitive_uvs", None).numpy()
normals = ans["primitive_normals"]

mustard-bottle is geom id 0
table is geom id 1


In [ ]:
valid = np.isfinite(t_hit).reshape(-1)

In [ ]:
########################
# Generate 3‑D vertices#
########################
rays_np = rays.numpy()  # (H*W,6)
origins = rays_np[..., :3]
dirs = rays_np[..., 3:]
noise = (depth_error_std * np.random.randn(*t_hit.shape)).astype(np.float32)
t_noisy = t_hit + noise
verts = origins + dirs * t_noisy[..., None]

In [ ]:
#############################
# Colour interpolation step #
#############################
vcols = np.full((*t_hit.shape, 3), 0.5)

# Assertions for bugfixing
assert bary_uv is not None
for mesh in meshlist:
    assert mesh.has_vertex_colors()

for id in range(len(meshlist)):
    rel_prim_ids = prim_ids[geom_ids==id]
    rel_bary_uv  =  bary_uv[geom_ids==id]
    vcols[geom_ids==id] = _interpolate_vertex_colors(meshlist[id], rel_prim_ids, rel_bary_uv)

In [ ]:
tris = triangulate_rgbd_grid(
    verts=verts,
    valid=valid,
    z=t_hit,
    obj_id=geom_ids,
    k=3.5,
    normals=None,                   # or your per-pixel normals (H,W,3)
    max_normal_angle_deg=None       # e.g. 60 to be stricter on folds
).astype(np.int32)

(240, 360, 3)
(86400,)
(240, 360)
(240, 360)

There are 83755 triangles


In [ ]:
mesh_out = o3d.geometry.TriangleMesh()
mesh_out.vertices = o3d.utility.Vector3dVector(verts.reshape(-1, 3))
mesh_out.triangles = o3d.utility.Vector3iVector(tris[:, [0,2,1]])

if vcols is not None:
    mesh_out.vertex_colors = o3d.utility.Vector3dVector(vcols.reshape(-1,3))

#############################
# Clean‑up / post‑processing#
#############################
mesh_out.remove_unreferenced_vertices()
mesh_out.remove_degenerate_triangles()
mesh_out.remove_duplicated_triangles()
mesh_out.remove_non_manifold_edges()
mesh_out.compute_vertex_normals()

######################
# Registration error #
######################
mesh_out.translate(tuple(np.random.randn(3) * translation_error_std))
R = mesh.get_rotation_matrix_from_xyz(
    tuple(np.random.randn(3) * np.deg2rad(rotation_error_std_degs))
)
mesh_out.rotate(R, center=mesh_out.get_center())

mesh = mesh_out

In [ ]:
o3d.visualization.draw_geometries([mesh])